In [24]:
import numpy as np
from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras import backend as K
from keras import layers
from keras.layers import BatchNormalization
from keras.layers import Activation

In [25]:
def Conv2d_bn(x,filters,kernel_size,padding='same',strides=(1, 1),normalizer=True,activation='relu',name=None):
    if name is not None:
        conv_name = name + '_conv'
        bn_name = name + '_bn'
        act_name = name + '_act'
    else:
        conv_name = None
        bn_name = None
        act_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = Conv2D(
            filters, kernel_size,
            strides=strides, padding=padding,
            use_bias=False, name=conv_name)(x)
    if normalizer:
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    if activation:
        x = Activation(activation, name=act_name)(x)
    return x

In [26]:
def InceptionV1_block(x, specs,channel_axis, name):
    (br0, br1, br2, br3) = specs   # ((64,), (96,128), (16,32), (32,))
    branch_0 = Conv2d_bn(x, br0[0], (1, 1), name=name+"_Branch_0")

    branch_1 = Conv2d_bn(x, br1[0], (1, 1), name=name+"_Branch_1")
    branch_1 = Conv2d_bn(branch_1, br1[1], (3, 3), name=name+"_Branch_1_1")

    branch_2 = Conv2d_bn(x, br2[0], (1, 1), name=name+"_Branch_2")
    branch_2 = Conv2d_bn(branch_2, br2[1], (5, 5), name=name+"_Branch_2_1")

    maxpool_3 = MaxPooling2D(pool_size = (3, 3), strides=(1, 1), padding='same', name = "maxpooling")(x)
    branch_3 = Conv2d_bn(maxpool_3, br3[0], (3, 3), name=name+"_Branch_3_1")
    

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name=name+"_Concatenated")
    return x

In [27]:
img_input = Input(shape=(224,224,1))
x=InceptionV1_block(img_input, ((64,), (96,128), (16,32), (32,)), 3, 'Block_1')
model = Model(img_input, x)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
Block_1_Branch_1_conv (Conv2D)  (None, 224, 224, 96) 96          input_12[0][0]                   
__________________________________________________________________________________________________
Block_1_Branch_2_conv (Conv2D)  (None, 224, 224, 16) 16          input_12[0][0]                   
__________________________________________________________________________________________________
Block_1_Branch_1_bn (BatchNorma (None, 224, 224, 96) 288         Block_1_Branch_1_conv[0][0]      
____________________________________________________________________________________________

In [37]:
def InceptionV3_block(x, specs,channel_axis, name):
    (br0, br1, br2, br3) = specs   # ((64,), (96,128), (16,32), (32,))
    branch_0 = Conv2d_bn(x, br0[0], (1, 1), name=name+"_Branch_0")

    branch_1 = Conv2d_bn(x, br1[0], (1, 1), name=name+"_Branch_1")
    branch_1 = Conv2d_bn(branch_1, br1[1], (1, 3), name=name+"_Branch_1_1")
    branch_1 = Conv2d_bn(branch_1, br1[1], (3, 1), name=name+"_Branch_1_2")

    branch_2 = Conv2d_bn(x, br2[0], (1, 1), name=name+"_Branch_2")
    branch_2 = Conv2d_bn(branch_2, br2[1], (1, 5), name=name+"_Branch_2_1")
    branch_2 = Conv2d_bn(branch_2, br2[1], (5, 1), name=name+"_Branch_2_2")
    
    maxpool_3 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same', name = 'maxpoolb3')(x)
    branch_3 = Conv2d_bn(maxpool_3, br3[0], (1, 1), name=name+"_Branch_3")

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name=name+"_Concatenated")
    return x

In [38]:
img_input = Input(shape=(224,224,1))
x=InceptionV3_block(img_input, ((64,), (96,128), (16,32), (32,)), 3, 'Block_1')
print(x)
model = Model(img_input, x)
model.summary()

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 256), dtype=tf.float32, name=None), name='Block_1_Concatenated/concat:0', description="created by layer 'Block_1_Concatenated'")
Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
Block_1_Branch_1_conv (Conv2D)  (None, 224, 224, 96) 96          input_18[0][0]                   
__________________________________________________________________________________________________
Block_1_Branch_2_conv (Conv2D)  (None, 224, 224, 16) 16          input_18[0][0]                   
__________________________________________________________________________________________________
Block